In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.6 MB/s eta 0:00:00


In [4]:
from torch_geometric.nn import GCNConv
from torch.distributions import Normal
import pandas as pd

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.autograd.set_detect_anomaly(False)

###############################################################
# Hyperparameters (Inputs)
###############################################################
NUM_REGIONS = 5
HIDDEN_DIM = 32
LATENT_DIM = 8
TIME_STEPS = 10   # fewer steps
EPOCHS = 500         # fewer epochs for demonstration
EPISODES_PER_EPOCH = 2
LEARNING_RATE = 1e-6
GAMMA = 0.99
bar_U = 0.0
MAX_RESOURCE = 1000.0

# Production parameters
alpha = 0.3
beta = 0.4
delta_k = 0.05
a = 0.01   # cost coefficient

# Shock
std_Z = 0.01

# Graph edges (chain)
GRAPH_EDGES = [(i, i+1) for i in range(NUM_REGIONS-1)]
edge_index = torch.tensor(GRAPH_EDGES, dtype=torch.long).t().contiguous().to(device)

# Loss Weights
WEIGHT_VAE = 1.0
WEIGHT_IC = 1.0
WEIGHT_IR = 1.0
WEIGHT_OSTROM = 1.0
WEIGHT_RESOURCE = 1.0
WEIGHT_POLICY = 1.0

# Monetary Policy Parameters

pi_star = 0.02  # target inflation
Y_star = 500.0  # arbitrary target output
phi_pi = 10.0
phi_y = 0.1

# Introduce population per region (just as a parameter, not fully utilized yet)
population = torch.tensor([100., 150., 120., 200., 180.], device=device)

In [8]:
# Functions
def g_func(R):
    K = MAX_RESOURCE
    r = 0.1
    return r * R * (1 - R/K)

def production(k_i, l_i, theta_i, Z):
    k_i = torch.clamp(k_i, min=1e-6)
    l_i = torch.clamp(l_i, min=1e-6)
    theta_i = torch.clamp(theta_i, min=0.1)
    return torch.exp(Z) * (theta_i**alpha) * (k_i**beta) * (l_i**(1-beta))

def utility_function(c_i, theta_i):
    c_i = torch.clamp(c_i, min=0)
    theta_i = torch.clamp(theta_i, min=0.1)
    return torch.sqrt(c_i * theta_i) - a * c_i

def resource_update(R, c_vector, x_vector):
    return R + g_func(R) - c_vector.sum() - x_vector.sum()

def capital_update(k_i, x_i):
    return (1 - delta_k)*k_i + x_i

def incentive_compatibility_loss(true_types, allocations):
    mean_type = true_types.mean()
    U_truthful = torch.tensor(0.0, device=device)
    U_deviant = torch.tensor(0.0, device=device)
    for i in range(NUM_REGIONS):
        U_truthful += utility_function(allocations[i], true_types[i])
        U_deviant += utility_function(allocations[i], mean_type)
    penalty = torch.clamp(U_deviant - U_truthful, min=0)
    return penalty

def individual_rationality_loss(avg_utilities, bar_U=bar_U):
    diff = avg_utilities - bar_U
    penalty = torch.mean(torch.clamp(-diff, min=0))
    return penalty

def ostrom_cooperation_loss(allocations_history):
    if len(allocations_history)<2:
        return torch.tensor(0.0, device=device)
    alloc_tensor = torch.stack(allocations_history, dim=0)
    var = alloc_tensor.var(dim=0).mean()
    return var

def resource_constraint_loss(R, c_alloc, x_alloc):
    cap = R + g_func(R)*10.0
    excess = torch.clamp(c_alloc.sum() + x_alloc.sum() - cap, min=0)
    return excess**2

###############################################################
# Models
###############################################################

class VAETypeInference(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAETypeInference, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, HIDDEN_DIM),
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, 2*latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, HIDDEN_DIM),
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, input_dim)
        )
        self.latent_dim = latent_dim

    def forward(self, x):
        stats = self.encoder(x)
        mu, logvar = stats[:,:self.latent_dim], stats[:,self.latent_dim:]
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = mu + eps*std
        recon = self.decoder(z)
        return recon, mu, logvar, z

    def loss_function(self, recon, x, mu, logvar):
        recon_loss = F.mse_loss(recon, x, reduction='mean')
        kl = -0.5*torch.mean(1+logvar - mu.pow(2)-logvar.exp())
        return recon_loss + kl

class DemandForecastModel(nn.Module):
    def __init__(self, input_dim=10, hidden_dim=16, num_layers=1):
        super(DemandForecastModel, self).__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.out = nn.Linear(hidden_dim, NUM_REGIONS)

    def forward(self, obs_history):
        output, hn = self.rnn(obs_history)
        pred = self.out(output[:,-1,:])
        return pred

class TransformerTemporalModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(TransformerTemporalModel, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=4)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        self.output_proj = nn.Linear(hidden_dim, hidden_dim)
    def forward(self, seq):
        x = self.input_proj(seq)
        x = x.permute(1,0,2)
        out = self.transformer(x)
        out = out.permute(1,0,2)
        return self.output_proj(out)

class GNNInteractionModel(nn.Module):
    def __init__(self, node_feature_dim):
        super(GNNInteractionModel, self).__init__()
        self.conv1 = GCNConv(node_feature_dim, HIDDEN_DIM)
        self.conv2 = GCNConv(HIDDEN_DIM, HIDDEN_DIM)
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, HIDDEN_DIM),
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, action_dim)
        )
    def forward(self, state):
        mean = self.fc(state)
        mean = torch.tanh(mean)*5.0
        mean = F.softplus(mean)
        mean = torch.clamp(mean, max=5.0)
        return mean

class ValueNetwork(nn.Module):
    def __init__(self, state_dim):
        super(ValueNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, HIDDEN_DIM),
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, 1)
        )
    def forward(self, state):
        return self.fc(state)

class SocialistEconomyEnv:
    def __init__(self):
        self.reset()
    def reset(self):
        self.R = torch.tensor(500.0, device=device)
        self.M = torch.tensor(1000.0, device=device)   # Money supply
        self.Z = torch.tensor(0.0, device=device)
        self.k = torch.ones(NUM_REGIONS, device=device)*50.0
        self.theta = torch.abs(torch.randn(NUM_REGIONS, device=device)) + 0.1
        self.region_obs = torch.randn(NUM_REGIONS, 10, device=device)
        self.t = 0
        self.done = False

        # NEW: We keep population here for possible future use
        self.population = population.clone()

        return self._get_state()
    def step(self, c_alloc, x_alloc, l_alloc):
        assert c_alloc.shape[0] == NUM_REGIONS
        assert x_alloc.shape[0] == NUM_REGIONS
        assert l_alloc.shape[0] == NUM_REGIONS

        y = torch.zeros(NUM_REGIONS, device=device)
        for i in range(NUM_REGIONS):
            y[i] = production(self.k[i], l_alloc[i], self.theta[i], self.Z)

        self.R = resource_update(self.R, c_alloc, x_alloc)
        self.k = capital_update(self.k, x_alloc)

        Z_shock = torch.randn(1, device=device)*std_Z
        self.Z = self.Z + Z_shock

        U = torch.zeros(NUM_REGIONS, device=device)
        for i in range(NUM_REGIONS):
            U[i] = utility_function(c_alloc[i], self.theta[i])

        # Compute inflation based on money supply changes
        # We'll implement a simple dynamic policy at the end of step:
        old_M = self.M.clone()
        Y = y.sum()
        # Price level P = M/Y
        P = self.M / Y
        # Let's define inflation as (M_{new} - M_old)/M_old after update
        # For now pi before update = 0, we adjust M:
        # DeltaM = phi_pi*(pi - pi_star) + phi_y*(Y - Y_star)
        # pi currently unknown, assume pi=0 for rule or stable scenario:
        # We'll just base policy on Y vs Y_star since pi not computed yet:
        pi_current = 0.0
        DeltaM = phi_pi*(pi_current - pi_star) + phi_y*(Y.item() - Y_star)
        self.M = self.M + DeltaM
        pi_new = (self.M - old_M)/old_M  # realized inflation

        self.t += 1
        if self.t >= TIME_STEPS:
            self.done = True

        with torch.no_grad():
            self.region_obs = self.region_obs + 0.01*torch.randn_like(self.region_obs)

        return U, self.done
    def _get_state(self):
        return (self.R, self.M, self.Z, self.k.mean(), self.theta.mean(), self.region_obs)

def compute_returns(rewards, gamma=GAMMA):
    returns = []
    R_ = 0
    for r in reversed(rewards):
        R_ = r + gamma*R_
        returns.insert(0,R_)
    return torch.tensor(returns, device=device, dtype=torch.float)

def main():
    vae = VAETypeInference(input_dim=10, latent_dim=LATENT_DIM).to(device)
    transformer = TransformerTemporalModel(input_dim=HIDDEN_DIM, hidden_dim=HIDDEN_DIM).to(device)
    gnn = GNNInteractionModel(node_feature_dim=HIDDEN_DIM).to(device)
    policy_net = PolicyNetwork(state_dim=HIDDEN_DIM + NUM_REGIONS*3, action_dim=NUM_REGIONS*3).to(device)
    value_net = ValueNetwork(state_dim=HIDDEN_DIM + NUM_REGIONS*3).to(device)
    demand_forecast = DemandForecastModel().to(device)

    optimizer = optim.Adam(list(vae.parameters())+
                           list(transformer.parameters())+
                           list(gnn.parameters())+
                           list(policy_net.parameters())+
                           list(value_net.parameters())+
                           list(demand_forecast.parameters()), lr=LEARNING_RATE)

    metrics_list = []

    # NEW: Data logging structures
    inputs_log = []
    params_log = []
    outputs_log = []

    for epoch in range(EPOCHS):
        for episode in range(EPISODES_PER_EPOCH):
            env = SocialistEconomyEnv()
            log_probs = []
            rewards = []
            allocations_history = []
            utilities_over_time = []

            done = False
            while not done:
                R, M, Z, mean_k, mean_theta, region_obs = env._get_state()
                recon, mu, logvar, z = vae(region_obs)
                vae_loss = vae.loss_function(recon, region_obs, mu, logvar)

                extra_feats = torch.randn(NUM_REGIONS, HIDDEN_DIM - LATENT_DIM, device=device)
                node_feats = torch.cat([z, extra_feats], dim=1)
                gnn_out = gnn(node_feats, edge_index)

                obs_for_forecast = region_obs.mean(dim=0, keepdim=True).unsqueeze(0)
                pred_demand = demand_forecast(obs_for_forecast).squeeze(0)

                global_vec = torch.cat([
                    gnn_out.mean(dim=0),
                    pred_demand,
                    torch.tensor([mean_k.item(), mean_theta.item(), R.item(), M.item(), Z.item()], device=device)
                ])

                needed_dim = HIDDEN_DIM + NUM_REGIONS*3
                if global_vec.shape[0] < needed_dim:
                    pad_size = needed_dim - global_vec.shape[0]
                    global_vec = torch.cat([global_vec, torch.zeros(pad_size, device=device)])
                global_state = global_vec.unsqueeze(0)

                mean_alloc = policy_net(global_state)
                c_alloc_mean = mean_alloc[0,:NUM_REGIONS]
                x_alloc_mean = mean_alloc[0,NUM_REGIONS:2*NUM_REGIONS]
                l_alloc_mean = mean_alloc[0,2*NUM_REGIONS:3*NUM_REGIONS]

                policy_dist_c = Normal(c_alloc_mean, 0.01*torch.ones_like(c_alloc_mean))
                policy_dist_x = Normal(x_alloc_mean, 0.01*torch.ones_like(x_alloc_mean))
                policy_dist_l = Normal(l_alloc_mean, 0.01*torch.ones_like(l_alloc_mean))

                max_alloc = env.R.item() / NUM_REGIONS / 10.0
                c_alloc = torch.clamp(policy_dist_c.sample(), min=0, max=max_alloc)
                x_alloc = torch.clamp(policy_dist_x.sample(), min=0, max=max_alloc)
                l_alloc = torch.clamp(policy_dist_l.sample(), min=1e-3, max=10.0)

                U, done = env.step(c_alloc, x_alloc, l_alloc)
                avg_U = U.mean().item()
                rewards.append(avg_U)

                log_p = (policy_dist_c.log_prob(c_alloc).sum() +
                         policy_dist_x.log_prob(x_alloc).sum() +
                         policy_dist_l.log_prob(l_alloc).sum())
                log_probs.append(log_p)

                allocations_history.append(c_alloc.detach())
                utilities_over_time.append(U.detach())

                # NEW: Logging inputs (state), parameters (allocations), and outputs (utility)
                inputs_log.append({
                    'Epoch': epoch+1,
                    'Episode': episode+1,
                    'Step': env.t,
                    'R': env.R.item(),
                    'M': env.M.item(),
                    'Z': env.Z.item(),
                    'mean_k': env.k.mean().item(),
                    'mean_theta': env.theta.mean().item()
                })

                params_log.append({
                    'Epoch': epoch+1,
                    'Episode': episode+1,
                    'Step': env.t,
                    'c_alloc_mean': c_alloc_mean.mean().item(),
                    'x_alloc_mean': x_alloc_mean.mean().item(),
                    'l_alloc_mean': l_alloc_mean.mean().item(),
                })

                outputs_log.append({
                    'Epoch': epoch+1,
                    'Episode': episode+1,
                    'Step': env.t,
                    'Utility': avg_U,
                    'c_alloc': c_alloc.mean().item(),
                    'x_alloc': x_alloc.mean().item(),
                    'l_alloc': l_alloc.mean().item()
                })

            returns = compute_returns(rewards, gamma=GAMMA)
            returns = (returns-returns.mean())/(returns.std()+1e-9)

            policy_loss = 0.0
            for lp, R_ in zip(log_probs, returns):
                policy_loss -= lp * R_

            utilities_tensor = torch.stack(utilities_over_time, dim=0)
            avg_utilities = utilities_tensor.mean(dim=0)
            mean_util = avg_utilities.mean().item()
            mean_alloc_val = torch.stack(allocations_history).mean(dim=0).mean().item()

            IC_loss = incentive_compatibility_loss(env.theta, allocations_history[-1])
            IR_loss = individual_rationality_loss(avg_utilities.mean())
            ostrom_loss_val = ostrom_cooperation_loss(allocations_history)
            resource_loss = resource_constraint_loss(env.R, allocations_history[-1], torch.zeros(NUM_REGIONS, device=device))

            total_loss = (vae_loss * WEIGHT_VAE +
                          IC_loss * WEIGHT_IC +
                          IR_loss * WEIGHT_IR +
                          ostrom_loss_val * WEIGHT_OSTROM +
                          resource_loss * WEIGHT_RESOURCE +
                          policy_loss * WEIGHT_POLICY)

            optimizer.zero_grad()
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(vae.parameters(), max_norm=1.0)
            torch.nn.utils.clip_grad_norm_(transformer.parameters(), max_norm=1.0)
            torch.nn.utils.clip_grad_norm_(gnn.parameters(), max_norm=1.0)
            torch.nn.utils.clip_grad_norm_(policy_net.parameters(), max_norm=1.0)
            torch.nn.utils.clip_grad_norm_(value_net.parameters(), max_norm=1.0)
            torch.nn.utils.clip_grad_norm_(demand_forecast.parameters(), max_norm=1.0)
            optimizer.step()

            # Print out inputs, parameters, outputs for debugging
            print(f"[Epoch {epoch+1}, Episode {episode+1}]")
            print("Inputs:")
            print(f" R={env.R.item():.4f}, M={env.M.item():.4f}, Z={env.Z.item():.4f}, mean_k={env.k.mean().item():.4f}, mean_theta={env.theta.mean().item():.4f}")
            print(f" region_obs mean={env.region_obs.mean().item():.4f}, std={env.region_obs.std().item():.4f}")

            print("Parameters (allocations):")
            print(f" c_alloc_mean={c_alloc_mean.mean().item():.4f}, x_alloc_mean={x_alloc_mean.mean().item():.4f}, l_alloc_mean={l_alloc_mean.mean().item():.4f}")
            print(f" c_alloc={c_alloc.mean().item():.4f}, x_alloc={x_alloc.mean().item():.4f}, l_alloc={l_alloc.mean().item():.4f}")

            print("Outputs:")
            print(f" Utility={mean_util:.4f}")
            print(f" Total Loss={total_loss.item():.4f}, VAE Loss={vae_loss.item():.4f}, IC Loss={IC_loss.item():.4f}, IR Loss={IR_loss.item():.4f}, Ostrom Loss={ostrom_loss_val.item():.4f}, Resource Loss={resource_loss.item():.4f}, Policy Loss={policy_loss.item():.4f}")
            print("-"*50)

            metrics_list.append({
                'Epoch': epoch+1,
                'Episode': episode+1,
                'R': env.R.item(),
                'M': env.M.item(),
                'Z': env.Z.item(),
                'Mean_k': env.k.mean().item(),
                'Mean_theta': env.theta.mean().item(),
                'region_obs_mean': env.region_obs.mean().item(),
                'region_obs_std': env.region_obs.std().item(),
                'c_alloc_mean': c_alloc_mean.mean().item(),
                'x_alloc_mean': x_alloc_mean.mean().item(),
                'l_alloc_mean': l_alloc_mean.mean().item(),
                'c_alloc': c_alloc.mean().item(),
                'x_alloc': x_alloc.mean().item(),
                'l_alloc': l_alloc.mean().item(),
                'Utility': mean_util,
                'Total Loss': total_loss.item(),
                'VAE Loss': vae_loss.item(),
                'IC Loss': IC_loss.item(),
                'IR Loss': IR_loss.item(),
                'Ostrom Loss': ostrom_loss_val.item(),
                'Resource Loss': resource_loss.item(),
                'Policy Loss': policy_loss.item()
            })

    # Save metrics to CSV
    metrics_df = pd.DataFrame(metrics_list)
    metrics_df.to_csv('training_metrics.csv', index=False)
    print("Metrics saved to training_metrics.csv")

    # NEW: Convert logs to DataFrames and save
    inputs_df = pd.DataFrame(inputs_log)
    params_df = pd.DataFrame(params_log)
    outputs_df = pd.DataFrame(outputs_log)

    inputs_df.to_csv('enhanced_inputs.csv', index=False)
    params_df.to_csv('enhanced_params.csv', index=False)
    outputs_df.to_csv('enhanced_outputs.csv', index=False)
    print("Enhanced logging data saved to CSV.")

In [9]:
if __name__ == "__main__":
    main()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Outputs:
 Utility=1.1186
 Total Loss=4.4124, VAE Loss=0.8326, IC Loss=0.0840, IR Loss=0.0000, Ostrom Loss=0.0003, Resource Loss=0.0000, Policy Loss=3.4956
--------------------------------------------------
[Epoch 274, Episode 1]
Inputs:
 R=471.6613, M=524.5308, Z=-0.0274, mean_k=50.8102, mean_theta=0.9150
 region_obs mean=0.0181, std=0.9980
Parameters (allocations):
 c_alloc_mean=2.9900, x_alloc_mean=2.8918, l_alloc_mean=2.0041
 c_alloc=2.9888, x_alloc=2.8908, l_alloc=2.0064
Outputs:
 Utility=1.2569
 Total Loss=-3.8096, VAE Loss=1.0388, IC Loss=0.1124, IR Loss=0.0000, Ostrom Loss=0.0001, Resource Loss=0.0000, Policy Loss=-4.9610
--------------------------------------------------
[Epoch 274, Episode 2]
Inputs:
 R=471.8211, M=523.5358, Z=-0.0476, mean_k=50.7528, mean_theta=1.5255
 region_obs mean=0.2024, std=0.9285
Parameters (allocations):
 c_alloc_mean=2.9903, x_alloc_mean=2.8680, l_alloc_mean=2.0041
 c_alloc=2.9860, x_alloc=2.8659, l_alloc=2.0067
Ou